In [1]:
import numpy as np 
import pandas as pd 
import tqdm
import string 

In [2]:
import seaborn as sns
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes

In [4]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

import nltk # for stopwords 
from nltk.corpus import stopwords
import gensim # for Word2Vec embeddings 
from sklearn.feature_extraction.text import CountVectorizer

from konlpy.tag import Mecab, Hannanum
from catboost import CatBoostClassifier

In [5]:
train = pd.read_csv('lgbm_train_df.csv') 
test = pd.read_csv('lgbm_test_df.csv')

In [6]:
x_train = train.drop(['n_id','info', 'title', 'content', 'title_tokenized', 'content_tokenized'], axis = 1) 
y_train = train['info'] 

x_train = np.asarray(x_train) 
y_train = np.asarray(y_train)

In [7]:
x_test = test.drop(['n_id','id','title', 'content', 'title_tokenized', 'content_tokenized'],axis=1) 
x_test = np.asarray(x_test)

In [8]:
x_train.shape, y_train.shape, x_test.shape

((118745, 63), (118745,), (142565, 63))

In [9]:
k = 5 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 990101) 
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)): 
    train_x, val_x = x_train[train_idx], x_train[val_idx]
    train_y, val_y = y_train[train_idx], y_train[val_idx] 
    
    model = CatBoostClassifier(iterations = 5000, 
                               learning_rate = 0.01, 
                               l2_leaf_reg = 3.5,
                               depth = 8, 
                               rsm = 0.98,
                               loss_function = 'Logloss',
                               eval_metric = 'Logloss',
                               use_best_model = True, 
                               random_seed = 960418)  
    
    model.fit(train_x, train_y, eval_set = (val_x, val_y)) 
    models.append(model)

0:	learn: 0.6709426	test: 0.6709548	best: 0.6709548 (0)	total: 120ms	remaining: 9m 58s
1:	learn: 0.6499645	test: 0.6499978	best: 0.6499978 (1)	total: 154ms	remaining: 6m 24s
2:	learn: 0.6300907	test: 0.6301293	best: 0.6301293 (2)	total: 194ms	remaining: 5m 23s
3:	learn: 0.6096021	test: 0.6096409	best: 0.6096409 (3)	total: 242ms	remaining: 5m 1s
4:	learn: 0.5903783	test: 0.5904730	best: 0.5904730 (4)	total: 288ms	remaining: 4m 47s
5:	learn: 0.5719405	test: 0.5720747	best: 0.5720747 (5)	total: 318ms	remaining: 4m 24s
6:	learn: 0.5528165	test: 0.5530014	best: 0.5530014 (6)	total: 355ms	remaining: 4m 13s
7:	learn: 0.5355496	test: 0.5357505	best: 0.5357505 (7)	total: 396ms	remaining: 4m 6s
8:	learn: 0.5188093	test: 0.5190262	best: 0.5190262 (8)	total: 424ms	remaining: 3m 54s
9:	learn: 0.5047031	test: 0.5048802	best: 0.5048802 (9)	total: 453ms	remaining: 3m 45s
10:	learn: 0.4914101	test: 0.4915550	best: 0.4915550 (10)	total: 479ms	remaining: 3m 37s
11:	learn: 0.4773196	test: 0.4775002	best: 

99:	learn: 0.1375460	test: 0.1382547	best: 0.1382547 (99)	total: 3.65s	remaining: 2m 58s
100:	learn: 0.1364900	test: 0.1372217	best: 0.1372217 (100)	total: 3.69s	remaining: 2m 58s
101:	learn: 0.1356274	test: 0.1363955	best: 0.1363955 (101)	total: 3.71s	remaining: 2m 58s
102:	learn: 0.1349249	test: 0.1356897	best: 0.1356897 (102)	total: 3.74s	remaining: 2m 57s
103:	learn: 0.1340610	test: 0.1348198	best: 0.1348198 (103)	total: 3.79s	remaining: 2m 58s
104:	learn: 0.1334971	test: 0.1342590	best: 0.1342590 (104)	total: 3.82s	remaining: 2m 58s
105:	learn: 0.1326689	test: 0.1334484	best: 0.1334484 (105)	total: 3.87s	remaining: 2m 58s
106:	learn: 0.1317145	test: 0.1325072	best: 0.1325072 (106)	total: 3.93s	remaining: 2m 59s
107:	learn: 0.1311007	test: 0.1319078	best: 0.1319078 (107)	total: 4.01s	remaining: 3m 1s
108:	learn: 0.1305092	test: 0.1313342	best: 0.1313342 (108)	total: 4.06s	remaining: 3m 2s
109:	learn: 0.1298164	test: 0.1306587	best: 0.1306587 (109)	total: 4.11s	remaining: 3m 2s
110:

190:	learn: 0.0989205	test: 0.1003482	best: 0.1003482 (190)	total: 6.67s	remaining: 2m 48s
191:	learn: 0.0987208	test: 0.1001606	best: 0.1001606 (191)	total: 6.7s	remaining: 2m 47s
192:	learn: 0.0984184	test: 0.0998666	best: 0.0998666 (192)	total: 6.74s	remaining: 2m 47s
193:	learn: 0.0981536	test: 0.0996129	best: 0.0996129 (193)	total: 6.76s	remaining: 2m 47s
194:	learn: 0.0979176	test: 0.0993883	best: 0.0993883 (194)	total: 6.8s	remaining: 2m 47s
195:	learn: 0.0976394	test: 0.0991291	best: 0.0991291 (195)	total: 6.83s	remaining: 2m 47s
196:	learn: 0.0974277	test: 0.0989364	best: 0.0989364 (196)	total: 6.86s	remaining: 2m 47s
197:	learn: 0.0971264	test: 0.0986431	best: 0.0986431 (197)	total: 6.89s	remaining: 2m 47s
198:	learn: 0.0968736	test: 0.0984071	best: 0.0984071 (198)	total: 6.92s	remaining: 2m 46s
199:	learn: 0.0966759	test: 0.0982146	best: 0.0982146 (199)	total: 6.95s	remaining: 2m 46s
200:	learn: 0.0964489	test: 0.0980000	best: 0.0980000 (200)	total: 6.98s	remaining: 2m 46s
2

285:	learn: 0.0820669	test: 0.0841943	best: 0.0841943 (285)	total: 9.7s	remaining: 2m 39s
286:	learn: 0.0819336	test: 0.0840582	best: 0.0840582 (286)	total: 9.74s	remaining: 2m 39s
287:	learn: 0.0818058	test: 0.0839413	best: 0.0839413 (287)	total: 9.79s	remaining: 2m 40s
288:	learn: 0.0816656	test: 0.0838090	best: 0.0838090 (288)	total: 9.82s	remaining: 2m 40s
289:	learn: 0.0815076	test: 0.0836579	best: 0.0836579 (289)	total: 9.86s	remaining: 2m 40s
290:	learn: 0.0814041	test: 0.0835568	best: 0.0835568 (290)	total: 9.91s	remaining: 2m 40s
291:	learn: 0.0813112	test: 0.0834788	best: 0.0834788 (291)	total: 9.94s	remaining: 2m 40s
292:	learn: 0.0812092	test: 0.0833742	best: 0.0833742 (292)	total: 9.98s	remaining: 2m 40s
293:	learn: 0.0810917	test: 0.0832706	best: 0.0832706 (293)	total: 10s	remaining: 2m 40s
294:	learn: 0.0809576	test: 0.0831416	best: 0.0831416 (294)	total: 10.1s	remaining: 2m 40s
295:	learn: 0.0808805	test: 0.0830673	best: 0.0830673 (295)	total: 10.1s	remaining: 2m 40s
29

376:	learn: 0.0722716	test: 0.0748972	best: 0.0748972 (376)	total: 13s	remaining: 2m 39s
377:	learn: 0.0721792	test: 0.0748105	best: 0.0748105 (377)	total: 13.1s	remaining: 2m 39s
378:	learn: 0.0721004	test: 0.0747460	best: 0.0747460 (378)	total: 13.1s	remaining: 2m 39s
379:	learn: 0.0719897	test: 0.0746434	best: 0.0746434 (379)	total: 13.1s	remaining: 2m 39s
380:	learn: 0.0719143	test: 0.0745775	best: 0.0745775 (380)	total: 13.2s	remaining: 2m 39s
381:	learn: 0.0718371	test: 0.0744953	best: 0.0744953 (381)	total: 13.2s	remaining: 2m 39s
382:	learn: 0.0717070	test: 0.0743797	best: 0.0743797 (382)	total: 13.2s	remaining: 2m 39s
383:	learn: 0.0716176	test: 0.0742994	best: 0.0742994 (383)	total: 13.3s	remaining: 2m 39s
384:	learn: 0.0715363	test: 0.0742129	best: 0.0742129 (384)	total: 13.3s	remaining: 2m 39s
385:	learn: 0.0714618	test: 0.0741492	best: 0.0741492 (385)	total: 13.3s	remaining: 2m 39s
386:	learn: 0.0713841	test: 0.0740766	best: 0.0740766 (386)	total: 13.4s	remaining: 2m 39s
3

...

4618:	learn: 0.0092021	test: 0.0251169	best: 0.0251169 (4618)	total: 2m 39s	remaining: 13.1s
4619:	learn: 0.0092002	test: 0.0251160	best: 0.0251160 (4619)	total: 2m 39s	remaining: 13.1s
4620:	learn: 0.0091969	test: 0.0251131	best: 0.0251131 (4620)	total: 2m 39s	remaining: 13.1s
4621:	learn: 0.0091955	test: 0.0251127	best: 0.0251127 (4621)	total: 2m 39s	remaining: 13s
4622:	learn: 0.0091928	test: 0.0251121	best: 0.0251121 (4622)	total: 2m 39s	remaining: 13s
4623:	learn: 0.0091928	test: 0.0251121	best: 0.0251121 (4622)	total: 2m 39s	remaining: 13s
4624:	learn: 0.0091892	test: 0.0251086	best: 0.0251086 (4624)	total: 2m 39s	remaining: 12.9s
4625:	learn: 0.0091877	test: 0.0251073	best: 0.0251073 (4625)	total: 2m 39s	remaining: 12.9s
4626:	learn: 0.0091819	test: 0.0251075	best: 0.0251073 (4625)	total: 2m 39s	remaining: 12.9s
4627:	learn: 0.0091819	test: 0.0251075	best: 0.0251073 (4625)	total: 2m 39s	remaining: 12.8s
4628:	learn: 0.0091803	test: 0.0251057	best: 0.0251057 (4628)	total: 2m 39s	

4710:	learn: 0.0090313	test: 0.0249852	best: 0.0249852 (4710)	total: 2m 42s	remaining: 9.95s
4711:	learn: 0.0090299	test: 0.0249821	best: 0.0249821 (4711)	total: 2m 42s	remaining: 9.91s
4712:	learn: 0.0090269	test: 0.0249820	best: 0.0249820 (4712)	total: 2m 42s	remaining: 9.88s
4713:	learn: 0.0090233	test: 0.0249759	best: 0.0249759 (4713)	total: 2m 42s	remaining: 9.85s
4714:	learn: 0.0090233	test: 0.0249759	best: 0.0249759 (4713)	total: 2m 42s	remaining: 9.81s
4715:	learn: 0.0090213	test: 0.0249730	best: 0.0249730 (4715)	total: 2m 42s	remaining: 9.78s
4716:	learn: 0.0090213	test: 0.0249730	best: 0.0249730 (4716)	total: 2m 42s	remaining: 9.74s
4717:	learn: 0.0090190	test: 0.0249717	best: 0.0249717 (4717)	total: 2m 42s	remaining: 9.71s
4718:	learn: 0.0090190	test: 0.0249717	best: 0.0249717 (4717)	total: 2m 42s	remaining: 9.67s
4719:	learn: 0.0090149	test: 0.0249695	best: 0.0249695 (4719)	total: 2m 42s	remaining: 9.64s
4720:	learn: 0.0090133	test: 0.0249682	best: 0.0249682 (4720)	total: 2

4801:	learn: 0.0088943	test: 0.0248796	best: 0.0248796 (4801)	total: 2m 44s	remaining: 6.8s
4802:	learn: 0.0088911	test: 0.0248767	best: 0.0248767 (4802)	total: 2m 44s	remaining: 6.76s
4803:	learn: 0.0088911	test: 0.0248767	best: 0.0248767 (4803)	total: 2m 44s	remaining: 6.73s
4804:	learn: 0.0088902	test: 0.0248763	best: 0.0248763 (4804)	total: 2m 44s	remaining: 6.7s
4805:	learn: 0.0088902	test: 0.0248763	best: 0.0248763 (4805)	total: 2m 45s	remaining: 6.66s
4806:	learn: 0.0088865	test: 0.0248703	best: 0.0248703 (4806)	total: 2m 45s	remaining: 6.63s
4807:	learn: 0.0088865	test: 0.0248703	best: 0.0248703 (4807)	total: 2m 45s	remaining: 6.59s
4808:	learn: 0.0088845	test: 0.0248688	best: 0.0248688 (4808)	total: 2m 45s	remaining: 6.56s
4809:	learn: 0.0088845	test: 0.0248688	best: 0.0248688 (4808)	total: 2m 45s	remaining: 6.52s
4810:	learn: 0.0088830	test: 0.0248691	best: 0.0248688 (4808)	total: 2m 45s	remaining: 6.49s
4811:	learn: 0.0088830	test: 0.0248691	best: 0.0248688 (4808)	total: 2m 

4890:	learn: 0.0087632	test: 0.0247693	best: 0.0247693 (4890)	total: 2m 47s	remaining: 3.73s
4891:	learn: 0.0087632	test: 0.0247693	best: 0.0247693 (4890)	total: 2m 47s	remaining: 3.7s
4892:	learn: 0.0087568	test: 0.0247586	best: 0.0247586 (4892)	total: 2m 47s	remaining: 3.66s
4893:	learn: 0.0087568	test: 0.0247586	best: 0.0247586 (4892)	total: 2m 47s	remaining: 3.63s
4894:	learn: 0.0087542	test: 0.0247560	best: 0.0247560 (4894)	total: 2m 47s	remaining: 3.6s
4895:	learn: 0.0087542	test: 0.0247560	best: 0.0247560 (4894)	total: 2m 47s	remaining: 3.56s
4896:	learn: 0.0087542	test: 0.0247560	best: 0.0247560 (4894)	total: 2m 47s	remaining: 3.53s
4897:	learn: 0.0087542	test: 0.0247560	best: 0.0247560 (4894)	total: 2m 47s	remaining: 3.49s
4898:	learn: 0.0087542	test: 0.0247561	best: 0.0247560 (4894)	total: 2m 47s	remaining: 3.46s
4899:	learn: 0.0087542	test: 0.0247561	best: 0.0247560 (4894)	total: 2m 47s	remaining: 3.42s
4900:	learn: 0.0087526	test: 0.0247551	best: 0.0247551 (4900)	total: 2m 

4981:	learn: 0.0086245	test: 0.0246731	best: 0.0246731 (4980)	total: 2m 50s	remaining: 615ms
4982:	learn: 0.0086225	test: 0.0246727	best: 0.0246727 (4982)	total: 2m 50s	remaining: 581ms
4983:	learn: 0.0086214	test: 0.0246724	best: 0.0246724 (4983)	total: 2m 50s	remaining: 547ms
4984:	learn: 0.0086193	test: 0.0246702	best: 0.0246702 (4984)	total: 2m 50s	remaining: 513ms
4985:	learn: 0.0086152	test: 0.0246707	best: 0.0246702 (4984)	total: 2m 50s	remaining: 478ms
4986:	learn: 0.0086134	test: 0.0246704	best: 0.0246702 (4984)	total: 2m 50s	remaining: 444ms
4987:	learn: 0.0086117	test: 0.0246689	best: 0.0246689 (4987)	total: 2m 50s	remaining: 410ms
4988:	learn: 0.0086117	test: 0.0246689	best: 0.0246689 (4987)	total: 2m 50s	remaining: 376ms
4989:	learn: 0.0086117	test: 0.0246689	best: 0.0246689 (4987)	total: 2m 50s	remaining: 342ms
4990:	learn: 0.0086117	test: 0.0246689	best: 0.0246689 (4987)	total: 2m 50s	remaining: 307ms
4991:	learn: 0.0086117	test: 0.0246689	best: 0.0246689 (4987)	total: 2

In [10]:
pred1 = models[0].predict_proba(x_test) 
pred2 = models[1].predict_proba(x_test) 
pred3 = models[2].predict_proba(x_test) 
pred4 = models[3].predict_proba(x_test) 
pred5 = models[4].predict_proba(x_test)

In [11]:
pred1

array([[9.94551075e-01, 5.44892522e-03],
       [9.99792499e-01, 2.07500730e-04],
       [9.99497835e-01, 5.02164728e-04],
       ...,
       [4.37436586e-05, 9.99956256e-01],
       [1.07351100e-04, 9.99892649e-01],
       [1.07530549e-04, 9.99892469e-01]])

In [12]:
np.save('catboost_logloss1.npy',pred1[:,1]) 
np.save('catboost_logloss2.npy',pred2[:,1]) 
np.save('catboost_logloss3.npy',pred3[:,1]) 
np.save('catboost_logloss4.npy',pred4[:,1]) 
np.save('catboost_logloss5.npy',pred5[:,1]) 
